In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#  split train and test in begining to avoid data leakage

In [ ]:
df = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/train.csv')
clm = ['TotRmsAbvGrd','GarageArea','1stFlrSF','WoodDeckSF', '2ndFlrSF', 'OpenPorchSF', 'HalfBath',
       'LotArea', 'BsmtFullBath', 'BsmtUnfSF', 'BedroomAbvGr', 'ScreenPorch',
       'PoolArea', 'MoSold', '3SsnPorch', 'BsmtFinSF2', 'BsmtHalfBath',
       'MiscVal', 'Id', 'LowQualFinSF', 'YrSold', 'OverallCond', 'MSSubClass',
       'EnclosedPorch', 'KitchenAbvGr']
#df = df.drop(columns = clm, axis=1)
dt = pd.read_csv('/kaggle/input/house-prices-advanced-regression-techniques/test.csv')
#dt = dt.drop(columns = clm, axis=1)
X_train, X_test, y_train, y_test = train_test_split(df.drop(['SalePrice'],axis=1), df['SalePrice'], test_size=0.2, random_state=0)

In [ ]:
dtrain = pd.concat([X_train,y_train],axis=1)
dtest = pd.concat([X_test,y_test],axis=1)
dtest['SalePrice']


In [ ]:
dtest = pd.concat([dtest,dt],axis=0)
dtest

In [ ]:
dtrain

In [ ]:
dtest

## Data preprocessing

for train

In [ ]:
missing_num = dtrain.select_dtypes(include=[np.number]).isnull().sum()
missing_num = missing_num[missing_num>0]
missing_obj = dtrain.select_dtypes(include=[np.object]).isnull().sum()
missing_obj = missing_obj[missing_obj>0]

In [ ]:
missing_num

In [ ]:
missing_num.sort_values().plot.bar()
plt.grid()

In [ ]:
missing_obj

In [ ]:
missing_obj.sort_values().plot.bar()
plt.grid()

In [ ]:
missing_obj = missing_obj[missing_obj> len(dtrain)/2.2]
dtrain.drop(missing_obj.index,axis=1,inplace = True)

In [ ]:
dtrain.select_dtypes(include=[np.object]).isnull().sum()

for test

In [ ]:
missing_num_test = dtest.select_dtypes(include=[np.number]).isnull().sum()
missing_num_test = missing_num_test[missing_num_test>0]
missing_obj_test = dtest.select_dtypes(include=[np.object]).isnull().sum()
missing_obj_test = missing_obj_test[missing_obj_test>0]

In [ ]:
missing_num_test

In [ ]:
missing_obj_test

In [ ]:
dtest.drop(missing_obj.index,axis=1,inplace = True)

In [ ]:
dtest.select_dtypes(include=[np.object]).isnull().sum()

## replacing nun value of numerical with mean

for train

In [ ]:
dtrain.select_dtypes(include=[np.object]).isnull().sum()

In [ ]:
missing_num.index

In [ ]:
for t in missing_num.index:
    dtrain[t] = dtrain[t].fillna(dtrain[t].mean())


In [ ]:
dtrain.select_dtypes(include=[np.number]).isnull().sum()

In [ ]:
dtrain.select_dtypes(include=[np.object]).isnull().sum()

In [ ]:
dtrain

for test

In [ ]:
for t in missing_num_test.index:
    dtest[t] = dtest[t].fillna(dtest[t].mean())

In [ ]:
dtest

replacing cardinal variables with mean

train

In [ ]:
cardi = dtrain.select_dtypes(include=[np.object]).isnull().sum()
cardi = cardi[cardi>0]
cardi.index

In [ ]:
k = dtrain['BsmtCond'].mode()
print(dict(k)[0])

In [ ]:
dtrain

In [ ]:
for t in cardi.index:
    k = dtrain[t].mode()
    dtrain[t].fillna(dict(k)[0],inplace = True)
    print(dtrain[t])

In [ ]:
dtrain[cardi.index].isnull().sum()

In [ ]:
cardi = dtest.select_dtypes(include=[np.object]).isnull().sum()
cardi = cardi[cardi>0]
cardi.index

In [ ]:
dtest[cardi.index].isnull().sum()

In [ ]:
for t in cardi.index:
    k = dtest[t].mode()
    dtest[t].fillna(dict(k)[0],inplace =True)
    print(dtest[t])

In [ ]:
dtest[cardi.index].isnull().sum()

In [ ]:
dtrain

In [ ]:
dtest

removing multi varriant variable

In [ ]:
numeric = dtrain.select_dtypes(include=[np.number])
numeric

In [ ]:
correlation = numeric.corr()
correlation_train =  correlation['SalePrice'].sort_values(ascending = False)


In [ ]:
correlation_train = correlation_train[correlation_train<0.32]
dtrain.drop(correlation_train.index, axis=1,inplace = True)
dtest.drop(correlation_train.index, axis=1, inplace = True)

In [ ]:
sns.heatmap(correlation,vmax=.8)

In [ ]:
k=11
cols =  correlation.nlargest(k,'SalePrice')['SalePrice'].index
print(cols)
cm = np.corrcoef(df[cols].values.T)
f , ax = plt.subplots(figsize = (14,12))
sns.heatmap(cm,vmax=.8,linewidths=.01,square =True, annot = True, cmap = 'viridis',
           linecolor="white",xticklabels = cols.values, annot_kws = {'size' : 10},yticklabels = cols.values)

In [ ]:
dtrain.select_dtypes(include=[np.number]).columns

removing outliyers

In [ ]:
sns.boxplot( x ='SalePrice' ,data = dtrain)

In [ ]:
q1 = dtrain['SalePrice'].quantile(.25)
q3 = dtrain['SalePrice'].quantile(.75)
qr = q3-q1
dtrain = dtrain.drop((dtrain['SalePrice'][dtrain['SalePrice']> q3 + 3 * qr]).index,axis=0)
dtrain['SalePrice']

In [ ]:
sns.boxplot( x ='SalePrice' ,data = dtrain)

for test

In [ ]:
dtest.shape

In [ ]:
Xytest = dtest[:292][:]
Xytest

In [ ]:
dtest.shape

In [ ]:
dtest['SalePrice']

In [ ]:
dtest.index

In [ ]:
dtest = dtest[292:1752][:]

In [ ]:
print(dtest['SalePrice'])
q1 = Xytest['SalePrice'].quantile(.25)
q3 = Xytest['SalePrice'].quantile(.75)
qr = q3-q1
Xytest.drop((Xytest['SalePrice'][Xytest['SalePrice']> q3 + 3 * qr]).index,axis=0, inplace =True)
Xytest['SalePrice'].shape

In [ ]:
dtest = pd.concat([Xytest,dtest],axis=0)
dtest[:288]['SalePrice']

In [ ]:
dtest.tail(50)

In [ ]:
dtrain

 for test 

In [ ]:
non_num = dtrain.select_dtypes(include = [np.object])
for t in non_num.columns:
    temp = dtrain[t].groupby(dtrain[t]).count()/len(dtrain)
    temp = temp[temp>0.01].index
    dtrain[t]=np.where(dtrain[t].isin(temp),dtrain[t], 'Rare_var')


In [ ]:
for t in non_num.columns:
    temp = dtest[t].groupby(dtest[t]).count()/len(dtest)
    temp = temp[temp>0.01].index
    dtest[t]=np.where(dtest[t].isin(temp),dtest[t], 'Rare_var')

In [ ]:
from sklearn.preprocessing import MinMaxScaler
numeric = dtrain.select_dtypes(include=[np.number])
numeric.drop('SalePrice',axis=1, inplace=True)
scaler = MinMaxScaler(copy=True, feature_range=(0,1))
scaled = scaler.fit_transform(dtrain[numeric.columns] )
print(scaled,dtrain)


In [ ]:
non_num = dtrain.select_dtypes(include=[np.object])

In [ ]:
dtrain_scaled = pd.concat([dtrain['SalePrice'].reset_index(drop=True),pd.DataFrame(scaled,columns = numeric.columns),dtrain[non_num.columns].dropna().reset_index(drop=True)],axis=1)

In [ ]:
dtrain_scaled

In [ ]:
numeric = dtest.select_dtypes(include=[np.number])
numeric.drop('SalePrice',axis=1, inplace=True)
scaler = MinMaxScaler(copy=True, feature_range=(0,1))
scaled = scaler.fit_transform(dtest[numeric.columns] )
print(scaled,dtest)

In [ ]:
dtest_scaled = pd.concat([dtest['SalePrice'].reset_index(drop=True),pd.DataFrame(scaled,columns = numeric.columns),dtest[non_num.columns].reset_index(drop=True)],axis=1)

In [ ]:
dtest_scaled

In [ ]:
coca = pd.concat([dtrain_scaled,dtest_scaled],axis=0)
coca.head(50)

In [ ]:
p = coca.select_dtypes(include=[np.number]).columns
temperal=[]

for x in p:
    if ("Sale" in x):
        temperal.append(x)
        
print(temperal) 


In [ ]:
f = pd.get_dummies(coca[coca.select_dtypes(include=[np.object]).columns])
f

In [ ]:
coca = pd.concat([coca.select_dtypes(include=[np.number]),f],axis=1)

In [ ]:
coca = coca.loc[:,~coca.columns.duplicated()]

In [ ]:
coca

In [ ]:
dtrain = coca[:1160][:]
dtest = coca[1160:2907][:]

In [ ]:
dtrain

In [ ]:
dtest['SalePrice']

Feature selection

In [ ]:
dtest[288:]

In [ ]:
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
import xgboost as xgb
from sklearn.model_selection import cross_val_score, GridSearchCV, KFold, RandomizedSearchCV
from scipy.stats import uniform, randint
from sklearn.metrics import auc, accuracy_score, confusion_matrix, mean_squared_error
from sklearn.feature_selection import SelectKBest, chi2

alpha = [0.05,0.01,0.1,0.2,0.3,0.5,0.7,1,2,5,50] 
for a in alpha:
    fm = SelectFromModel(Lasso(alpha = a ,random_state =0))
    fm.fit(dtrain.drop('SalePrice',axis=1), dtrain['SalePrice'])
    print("Total no of features selected : {}  and alpha is :".format(fm.get_support().sum()),a )
    x = dtrain.drop('SalePrice',axis=1)
    feature = x.columns[(fm.get_support())]
    dtrain = pd.concat([dtrain[feature], dtrain['SalePrice']],axis=1)
    dtest = pd.concat([dtest[feature],dtest['SalePrice']],axis=1)
    xgb_model = xgb.XGBRegressor()
    xgb_model.fit(dtrain.drop('SalePrice',axis=1), dtrain['SalePrice'])
    print("train score : {}".format(xgb_model.score(dtrain.drop('SalePrice',axis=1), dtrain['SalePrice'])))
    print("\ntest score : {}".format(xgb_model.score(dtest[:288][:].drop('SalePrice',axis=1), dtest[:288][:]['SalePrice'])))
    #report_best_scores(search.cv_results_, 1)
    #f = xgp_model.predict(dtest[288:][:].drop('SalePrice',axis=1))      
    dataf = pd.DataFrame(xgb_model.predict(dtest[288:][:].drop('SalePrice',axis=1)), columns = ['SalePrice'])
    dataf.index += 1461
    dataf.to_csv("submission.csv")
    
    

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV 
from sklearn.model_selection import train_test_split
from sklearn.model_selection import RandomizedSearchCV
from sklearn import linear_model

In [ ]:
X_train1 =dtrain.drop('SalePrice',axis=1)
y_train1 = dtrain['SalePrice']

In [ ]:
X_train1 

In [ ]:
fscore = pd.concat([dfcolumns,dfscore],axis=1)
fscore.columns = ['feat','Score']
fscore

In [ ]:
X = fscore.nlargest(50,'Score')
X

In [ ]:
graphleg=[]
graphxg=[]

In [ ]:
X

In [ ]:
for k in range(10,200):
    X_train1 =dtrain.drop('SalePrice',axis=1)
    y_train1 = dtrain['SalePrice']
    X_new = SelectKBest(chi2, k).fit(X_train1, y_train1)
    dfscore = pd.DataFrame(X_new.scores_)
    dfcolumns = pd.DataFrame(X_train1.columns)
    fscore = pd.concat([dfcolumns,dfscore],axis=1)
    fscore.columns = ['feat','Score']
    X = fscore.nlargest(k,'Score')
    X_train1 = dtrain[X['feat']]
    y_train1 = dtrain['SalePrice']
    X_test1 = dtest[:288][X['feat']]
    y_test1 = dtest[:288]['SalePrice']
    Legreg = LinearRegression()
    adabost = AdaBoostRegressor(random_state =0)
    rndfor = RandomForestRegressor(random_state =0)
    xgb_model = xgb.XGBRegressor()
    clf2 = linear_model.SGDRegressor(max_iter=1000, tol=1e-3)
    Legreg.fit(X_train1,y_train1)
    adabost.fit(X_train1,y_train1)
    rndfor.fit(X_train1,y_train1)
    clf2.fit(X_train1,y_train1)
    xgb_model.fit(X_train1,y_train1)
    print(k)
    print("\n")
    print("TRAIN:\nLegreg.score : {}\nadabost.score : {}\nrndfor.score : {}\nclf2.score : {}\nxgb_model.score : {}".format(Legreg.score(X_train1,y_train1)
    ,adabost.score(X_train1,y_train1)
    ,rndfor.score(X_train1,y_train1),clf2.score(X_train1,y_train1),xgb_model.score(X_train1,y_train1)))


    print("TEST:\nLegreg.score : {}\nadabost.score : {}\nrndfor.score : {}\nclf2.score : {}\nxgb_model.score : {}".format(Legreg.score(X_test1,y_test1)
    ,adabost.score(X_test1,y_test1)
    ,rndfor.score(X_test1,y_test1),clf2.score(X_test1,y_test1),xgb_model.score(X_test1,y_test1)))
    graphleg.append(rndfor.score(X_test1,y_test1))
    graphxg.append(xgb_model.score(X_test1,y_test1))


In [ ]:
max(graphxg)

In [ ]:
max(graphleg)

In [ ]:
fig = sns.lineplot(x=range(10,10 +len(graphxg)), y= graphleg)
fig.set(xlabel = 'K',ylabel = 'TestScore RANDOM FOREST')
plt.show()

In [ ]:
fig = sns.lineplot(x=range(10,10 +len(graphxg)), y= graphxg)
fig.set(xlabel = 'K',ylabel = 'TestScore  XGBOOST')
plt.show()

In [ ]:
    k=53
    X = fscore.nlargest(k,'Score')
    X_train1 = dtrain[X['feat']]
    y_train1 = dtrain['SalePrice']
    X_test1 = dtest[:288][:][X['feat']]
    y_test1 = dtest[:288][:]['SalePrice']
    Legreg = LinearRegression()
    adabost = AdaBoostRegressor(random_state =0)
    rndfor = RandomForestRegressor(random_state =0)
    xgb_model = xgb.XGBRegressor()
    clf2 = linear_model.SGDRegressor(max_iter=1000, tol=1e-3)
    Legreg.fit(X_train1,y_train1)
    adabost.fit(X_train1,y_train1)
    rndfor.fit(X_train1,y_train1)
    clf2.fit(X_train1,y_train1)
    xgb_model.fit(X_train1,y_train1)
    print(k)
    print("\n")
    print("TRAIN:\nLegreg.score : {}\nadabost.score : {}\nrndfor.score : {}\nclf2.score : {}\nxgb_model.score : {}".format(Legreg.score(X_train1,y_train1)
    ,adabost.score(X_train1,y_train1)
    ,rndfor.score(X_train1,y_train1),clf2.score(X_train1,y_train1),xgb_model.score(X_train1,y_train1)))


    print("TEST:\nLegreg.score : {}\nadabost.score : {}\nrndfor.score : {}\nclf2.score : {}\nxgb_model.score : {}".format(Legreg.score(X_test1,y_test1)
    ,adabost.score(X_test1,y_test1)
    ,rndfor.score(X_test1,y_test1),clf2.score(X_test1,y_test1),xgb_model.score(X_test1,y_test1)))
    graphleg.append(Legreg.score(X_test1,y_test1))
    graphxg.append(xgb_model.score(X_test1,y_test1))

In [ ]:
X_train1

In [ ]:
y_train1

In [ ]:
X_test1

In [ ]:
y_test1

In [ ]:
dtest[288:][X['feat']].drop('SalePrice',axis=1, inplace=True)

In [ ]:
dataf = pd.DataFrame(Legreg.predict(dtest[288:][X['feat']]), columns = ['SalePrice'])
dataf.index += 1461
dataf.to_csv("submission.csv")
dataf = pd.DataFrame(xgb_model.predict(dtest[288:][X['feat']]), columns = ['SalePrice'])
dataf.index += 1461
dataf.to_csv("submission2.csv")
